# Get Places

We will study the layout of the output from the API

In [61]:
import pandas as pd
import json
import folium

In [ ]:
import os

In [62]:
# Load json 
walk_path: str = "../tmp/"
samples: list = list()
for _,_, files in os.walk(walk_path):
    for file in files:
        with open(example_path) as f:
            loaded_var: dict = json.load(f)
        results: list[dict] = loaded_var['results']
        samples.append(results)
    

In [60]:
[len(i) for i in samples]

[10, 10, 10, 10, 10]

In [36]:
df = pd.json_normalize(
    results,
    sep='_',  # Use underscore to join keys
    max_level=0  # Avoid flattening hours_popular/regular lists into columns
)

In [63]:
df = pd.DataFrame(results)

In [65]:
df

,fsq_id,categories,chains,closed_bucket,distance,geocodes,link,location,name,related_places,timezone
0,148f444f6fed4e9fcaaec895,[],[],LikelyOpen,52,"{'drop_off': {'latitude': 51.18066, 'longitude...",/v3/places/148f444f6fed4e9fcaaec895,"{'address': 'Glasstraat 37', 'country': 'BE', ...",Mb Industrial Piping,{},Europe/Brussels
